# FAH GPU performance analysis example

**DISCLAIMER: the GPU benchmark data in this repo is experimental and not final. For now, the code and data presented here is intended as an example of parsing performance data from fah-core logs, and of the types of analysis that might be useful to automate intelligent assignment of work units to specific devices.**

Here we analyze performance data (`ns_per_day`) parsed from log files created by `fah-core` for the project `PROJ17101`. This project is intended to be a benchmark suite and contains 15 runs representative of the range of workloads run on FAH today.

In [1]:
import glob
import os
import altair as alt
import numpy as np
import pandas as pd
import seaborn as sns
from fah_log_parser import ParseError, parse, science_log
from fah_log_parser.util.pandas import parse_project_logs

/home/hmacdope/anaconda3/envs/openmm-core-v2/lib/python3.9/site-packages/pydantic-2.0b2-py3.9.egg/pydantic/_internal/_config.py:237: UserWarning: Valid config keys have changed in V2:
* 'allow_mutation' has been removed
  warnings.warn(message, UserWarning)


## Parsing a single `science.log` file

In [2]:
log = parse(science_log, "../PROJ17113/RUN0/CLONE0/results0/science.log") # linux 


In [3]:
log = parse(science_log, "../PROJ17113/RUN12/CLONE172/results0/science.log") 

In [4]:
log.core_header.platform

'win32 10'

In [5]:
platform, device = log.get_active_device()

In [6]:
platform

PlatformInfo(profile='FULL_PROFILE', version='OpenCL 3.0 CUDA 11.7.101', name='NVIDIA CUDA', vendor='NVIDIA Corporation')

In [7]:
device

Device(name='NVIDIA GeForce RTX 3060', vendor='NVIDIA Corporation', version='OpenCL 3.0 CUDA', driver_version='516.94')

In [8]:
log.core_log.average_perf_ns_day

137.759

## Parsing all logs in a project

In [9]:
if os.path.exists("checkpoint"):
    data = pd.read_feather("checkpoint")
else:
    data = parse_project_logs("../PROJ17113/", sample=10)
    data.reset_index.to_feather("checkpoint")
    
data.info()

3201 files found in path ../PROJ17113/


/home/hmacdope/anaconda3/envs/openmm-core-v2/lib/python3.9/site-packages/rich-13.4.2-py3.9.egg/rich/live.py:231: 
UserWarning: install "ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

AttributeError: 'function' object has no attribute 'to_feather'

In [ ]:
df = (data
 .set_index(['run', 'clone', 'gen', 'device_name'])
 ['perf_ns_per_day']
 .groupby('device_name').filter(lambda x: len(x) > 10)
 .groupby('run').transform(lambda x: (x - x.mean()) / x.std()).rename('z_score_by_run')
 .groupby('device_name').mean()
).reset_index()

alt.Chart(df).mark_bar().encode(
    x=alt.X("z_score_by_run:Q", aggregate='mean'),
    y=alt.Y("device_name:N", sort='-x')
)

In [ ]:
df = (data
 .set_index(['run', 'clone', 'gen', 'device_name'])
 ['perf_ns_per_day']
 .groupby(['run', 'device_name']).filter(lambda x: len(x) > 10)
 .groupby('device_name').filter(lambda x: len(x.groupby('run')) == 15)
 .groupby(['run', 'device_name']).mean()
 .rename('mean_ns_per_day')
).reset_index()

alt.Chart(df).mark_point(filled=True).encode(
    x=alt.X('mean_ns_per_day:Q', scale=alt.Scale(type='log')),
    y=alt.Y('device_name:O', sort='-x'),
    color='run:N',
)

In [ ]:
sns.clustermap(
    data
    .groupby(['run', 'device_name']).filter(lambda x: len(x) > 10)
    .groupby('device_name').filter(lambda x: len(x.groupby('run')) == 15)
    .groupby(['run', 'device_name'])
    ['perf_ns_per_day']
    .mean()
    .pipe(lambda x: np.log(x))
    .unstack('run'))